In [ ]:
# %%
# filename: optim_backtestingpy_sma_optimization.ipynb

# --- 1. Imports and Setup ---
import pandas as pd
import pandas_ta as ta  # We'll use pandas-ta for the indicator calculation inside the strategy
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import EURUSD

# --- 2. Load Sample Data ---
print("Loading built-in EURUSD sample data...")
data = EURUSD.copy()
print("Data loaded successfully.")

In [ ]:
# %%
# --- 3. The Parameterized Strategy Class ---

class SmaCrossOptimize(Strategy):
    # --- Parameters for Optimization ---
    # These values will be replaced by the optimizer.
    # The values here are just defaults for a single run.
    fast_period = 10
    slow_period = 40

    def init(self):
        # In init(), we define the indicators. Since the periods are dynamic,
        # we calculate them here. self.I() is smart and caches the results.
        # We use pandas_ta for its clean syntax.
        self.fast_sma = self.I(ta.sma, pd.Series(self.data.Close), length=self.fast_period)
        self.slow_sma = self.I(ta.sma, pd.Series(self.data.Close), length=self.slow_period)

    def next(self):
        # The same trading logic as before
        if crossover(self.fast_sma, self.slow_sma):
            if not self.position:
                self.buy()
        elif crossover(self.slow_sma, self.fast_sma):
            self.position.close()

In [ ]:
# %%
# --- 4. Running the Optimization ---
print("\n--- Starting Grid Search Optimization ---")

bt = Backtest(data, SmaCrossOptimize, cash=10000, commission=.002)

# The optimize method performs a grid search.
# We store the results, which include the heatmap data, in a variable.
# return_heatmap=True is the default, so we don't need to specify it, but it's good to be aware of.
stats, heatmap = bt.optimize(
    fast_period=range(10, 31, 5),
    slow_period=range(40, 81, 10),
    maximize='Sharpe Ratio',
    constraint=lambda p: p.fast_period < p.slow_period,
    return_heatmap=True  # Explicitly ask for the heatmap data
)

print("\n--- Optimization Results ---")
print("Best stats found:")
print(stats)

print("\nBest parameters found:")
print(stats._strategy)

In [ ]:
# %%
# --- 5. Visualizing the Optimization Results ---
from backtesting.lib import plot_heatmaps

print("\n--- Plotting Optimization Heatmap ---")

# The plot_heatmaps function is imported directly from backtesting.lib
# and takes the heatmap data returned by the optimize() method.
plot_heatmaps(heatmap, agg='mean')

In [ ]:
# %%
# --- 6. Walk-Forward Analysis (WFA) ---
# This is a more robust way to test our strategy. We optimize on a training set
# and validate on an unseen testing set, moving this window through time.

print("\n--- Starting Walk-Forward Analysis ---")

# --- WFA Configuration ---
n_train_bars = 24 * 30 * 3  # Approx. 3 months
n_test_bars = 24 * 30 * 1   # Approx. 1 month
step = n_test_bars

oos_results = []

# The main loop for walking forward
for i in range(n_train_bars, len(data) - n_test_bars, step):
    train_data = data.iloc[i - n_train_bars : i]
    test_data = data.iloc[i : i + n_test_bars]

    print(f"\nTraining on data from {train_data.index[0]} to {train_data.index[-1]}")
    print(f"Testing on data from {test_data.index[0]} to {test_data.index[-1]}")

    # --- In-Sample Optimization ---
    # For the training part, finalizing trades is less critical, but we can add it for consistency.
    bt_train = Backtest(train_data, SmaCrossOptimize, cash=10000, commission=.002, finalize_trades=True)

    stats_train, _ = bt_train.optimize(
        fast_period=range(10, 31, 5),
        slow_period=range(40, 81, 10),
        maximize='Sharpe Ratio',
        constraint=lambda p: p.fast_period < p.slow_period,
        return_heatmap=True
    )

    best_params = stats_train._strategy
    print(f"Best params found in-sample: {best_params.fast_period}/{best_params.slow_period}")

    # --- Out-of-Sample Validation ---
    # CORRECTED: Add finalize_trades=True to get more accurate OOS stats.
    bt_test = Backtest(test_data, SmaCrossOptimize, cash=10000, commission=.002, finalize_trades=True)

    stats_test = bt_test.run(
        fast_period=best_params.fast_period,
        slow_period=best_params.slow_period
    )

    oos_results.append(stats_test)


# --- 7. Analyze WFA Results ---
print("\n\n--- Walk-Forward Analysis Final Results ---")

if oos_results:
    wfa_df = pd.concat([res.to_frame(f"Period_{i}") for i, res in enumerate(oos_results)], axis=1).T

    print("Performance of each Out-of-Sample period:")
    print(wfa_df[['Return [%]', 'Sharpe Ratio', 'Win Rate [%]', '# Trades']])

    print("\nAverage performance across all OOS periods:")
    print(wfa_df[['Return [%]', 'Sharpe Ratio', 'Win Rate [%]']].mean())
else:
    print("No out-of-sample periods were successfully tested.")